In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch.optim as optim
import os
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = f.max_pool2d(x, 2)
        x = f.relu(self.conv2(x))
        x = f.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
NORMALIZE = False

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_home = 'F:\\work'
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root=os.path.join(data_home, 'dataset/CIFAR10'), train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=2)

# CSA 

In [5]:
class Loss_cost_sensitive(nn.Module):
    def __init__(self):
        super(Loss_cost_sensitive, self).__init__()
    def forward(self, data, target, c):
        # data: 模型输出
        # target: 标签
        # c: 代价矩阵
        l1 = F.cross_entropy(data, target, reduction='mean')
        p = F.softmax(data, 1)
        
        cost_sentive = c[:,target]
        cost_sentive = cost_sentive.T
        l2 = p.mul(cost_sentive)
        l2 = l2.sum(1).mean()
        return l1+l2
    
def my_fgsm(input, labels, model, criterion, epsilon, device, c=None):
    assert isinstance(model, torch.nn.Module), "Input parameter model is not nn.Module. Check the model"
    assert isinstance(criterion, torch.nn.Module), "Input parameter criterion is no Loss. Check the criterion"
    assert (0 <= epsilon <= 1), "episilon must be 0 <= epsilon <= 1"

    # For calculating gradient
    input_for_gradient = Variable(input, requires_grad=True).to(device)
    out = model(input_for_gradient)
    if c==None:
        loss = criterion(out, Variable(labels))
    else:
        loss = criterion(out, Variable(labels), c)

    # Calculate gradient
    loss.backward()

    # Calculate sign of gradient
    signs = torch.sign(input_for_gradient.grad.data)

    # Add
    input_for_gradient.data = input_for_gradient.data + (epsilon * signs)

    return input_for_gradient, signs

def get_cost_matric(i_label):
    # 生成保护i_label类的代价矩阵
    # i_label：受保护类
    C = torch.ones(10,10)
    C[i_label,:] = 5
    C[:,i_label] = 5
    C = C - torch.diag(C.diag())
    return C

In [6]:
model_adv = LeNet()
if NORMALIZE:
    model_adv.load_state_dict(torch.load('../model/Lenet_CIFAR_ADV.pt'))
else:
    model_adv.load_state_dict(torch.load('../model/Lenet_CIFAR_ADV.pt'))

In [ ]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon = 0.3
criterion = Loss_cost_sensitive()
if NORMALIZE:
    model_adv_path = '../model/Lenet_CIFAR_ADV.pt'
else:
    model_adv_path = '../model/Lenet_CIFAR_ADV.pt'

# 循环 对每一个类分别进行保护
for i_label in range(10):
    # 读取预训练模型
    model_cost_sensitive = LeNet()
    model_cost_sensitive.load_state_dict(torch.load(model_adv_path))
    print('load model for initialization: {}'.format(model_adv_path))
    model_cost_sensitive = model_cost_sensitive.to(DEVICE)
    
    optimizer = torch.optim.SGD(params=model_cost_sensitive.parameters(), lr=0.0001, momentum=0.5)
    schedule = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.5)
    
    C = get_cost_matric(i_label)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    
    LABEL = 'Protect Label ' + str(i_label)
    
    # 开始训练
    print('开始训练')
    for epoch in range(50):
        count = 0
        model_cost_sensitive.train()
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_cost_sensitive, criterion, epsilon, DEVICE, C)
    
            optimizer.zero_grad()
            output = model_cost_sensitive(data)
            loss = criterion(output, target, C)
            
            loss.backward()
            optimizer.step()
            schedule.step()
            
            count += len(data)
            print('\r {}|{}'.format(count, len(train_loader.dataset)), end='')
        
        # 测试
        correct = 0
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
#             
            data, sign = my_fgsm(data, target, model_cost_sensitive, criterion, epsilon, DEVICE, C)
            output = model_cost_sensitive(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        print('epoch: {}, test correct: {}'.format(epoch,correct/len(test_loader.dataset)))
    
    # 保存模型
    if NORMALIZE:
        model_save_path = '../model/LeNet_CIFAR_adv_cost_sensitive_' + str(i_label) +'.pt'
    else:
        model_save_path = '../model/LeNet_CIFAR_unnormalized_adv_cost_sensitive_' + str(i_label) +'.pt'
    torch.save(model_cost_sensitive.state_dict(), model_save_path)
    
    
    # 训练结束
#     比较结果

    ## 对 model_cost_sensitive 的评估
    print('对 model_cost_sensitive 的评估')
    results_info = {}
    model_cost_sensitive.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0
        info_arr = torch.tensor([])
        info_arr = info_arr.to(DEVICE)

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_cost_sensitive, criterion, epsilon, DEVICE, C)
            output = model_cost_sensitive(data)

            # 计算熵
            tmp_output = torch.softmax(output, dim=1)
            info_entropy = -tmp_output * torch.log2(tmp_output)
            info_entropy = info_entropy.sum(dim=1)
            info_arr = torch.cat((info_arr, info_entropy))

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count, info_arr.detach().cpu().numpy()]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
    ## 对 model_adv 的评估
    print('对 model_adv 的评估')
    model_adv.eval()
    images_targets = {}
    criterion_adv = nn.CrossEntropyLoss()
    for special_index in range(10):
        count = 0
        correct = 0
        info_arr = torch.tensor([])
        info_arr = info_arr.to(DEVICE)

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_adv, criterion_adv, epsilon, DEVICE)
            output = model_adv(data)

            # 计算熵
            tmp_output = torch.softmax(output, dim=1)
            info_entropy = -tmp_output * torch.log2(tmp_output)
            info_entropy = info_entropy.sum(dim=1)
            info_arr = torch.cat((info_arr, info_entropy))

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count, info_arr.detach().cpu().numpy()]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
    # 记录结果
    results_infos[i_label] = results_info
    
    break

load model for initialization: ../model/Lenet_CIFAR_ADV.pt
protect label: 0
load cost matric: 
tensor([[0., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
        [5., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [5., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
        [5., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
        [5., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
        [5., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
        [5., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
        [5., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [5., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [5., 1., 1., 1., 1., 1., 1., 1., 1., 0.]])
开始训练
 50000|50000epoch: 0, test correct: 0.7903
 1856|50000